In [1]:
import pandas as pd, numpy as np
import os
from glob import glob as g

In [2]:
def split_dataframe(df, length):
    # List to hold the split DataFrames
    split_dfs = []
    
    # Calculate the number of splits needed
    num_splits = len(df) // length + (1 if len(df) % length != 0 else 0)
    
    for i in range(num_splits):
        # Calculate start and end indices for the current split
        start_idx = i * length
        end_idx = start_idx + length
        
        # Slice the DataFrame and add it to the list
        split_dfs.append(df.iloc[start_idx:end_idx])
    print(f"Created {len(split_dfs)} chunks.")
    return split_dfs

In [5]:
## set source 
source = r'C:\Users\AbhishekSrivastava\Desktop\Workflow\Python\Dev\Medical Devices Registry\MDR Scripts\CDSCO IN'
file = "\CDSCO_IN - Approved Medical Devices 4Jan24.xlsx"

In [7]:
payload = pd.read_excel(source+file)
payload.shape

(10954, 7)

In [27]:
payload['S. No.'].unique()

array([    1,     2,     3, ..., 10952, 10953, 10954], dtype=int64)

In [9]:
cols = payload.columns
# cols_en = [translate_to_en(x) for x in cols]
# cols_en
cols

Index(['Unnamed: 0', 'S. No.', 'License Number', 'License Holder Name',
       'Device Name(Brand Name)-Intended Use', 'Approving Authority',
       'Device Class'],
      dtype='object')

In [101]:
grill = payload.sample(3)
grill

,Unnamed: 0,S. No.,License Number,License Holder Name,Device Name(Brand Name)-Intended Use,Approving Authority,Device Class
3185,85,3186,IMP/MD/2019/000182,Johnson & Johnson Private Limited,Endoscopic Applicator(SURGICEL Endoscopic Appl...,CDSCO,Class B
787,87,788,IMP/IVD/2020/000097,ANALYTICAL AUTOMATION ( INDIA) PVT. LTD.,"Cleaner(HC500 Cleaner, HC300 Cleaner)-Cleaner ...",CDSCO,Class B
529,29,530,IMP/IVD/2019/000376,Beckman Coulter India Private Limited Hisaria ...,Access PCT (Reagent Pack)(Access PCT (Reagent ...,CDSCO,Class B


In [71]:
# Regular expression pattern to extract Device Name, Brand Name, and Intended Use
pattern = r'(.+?)\((.+?)\)-(.+),'

In [ ]:
(.+?)\((.+?)\)-(.+?)

In [123]:
chunk = payload[['S. No.','Device Name(Brand Name)-Intended Use']]
chunk

,S. No.,Device Name(Brand Name)-Intended Use
0,1,Urethral Catheter(GPC Urethral Catheter)-It is...
1,2,Natural Rubber Latex Male Condoms(ULINZI - For...
2,3,25-OH VITAMIN D ELISA-The Product is intended ...
3,4,ENZYMATIC ASSAY FOR THE QUANTITATIVE DETERMINA...
4,5,Surgical Suture(Dynalon)-Use in general soft t...
...,...,...
10949,10950,HIV Gen.4 Microwell ELISA(VDx)-HIV Gen. 4 Micr...
10950,10951,Locking Bolts 7.5mm Cannulated (Self Tapping)(...
10951,10952,Total Hip Replacement System(Pl refer Accessor...
10952,10953,ADHESIVE DRESSING BANDAGES FABRIC(CERDAK)-Exud...


In [125]:
# Original pattern with capturing groups
pattern = r'(.+?)\((.+?)\)-(.+?)(?=(\.,|$))'

# Modified pattern for splitting - use non-capturing groups (?:...) for parts you don't want to remove
split_pattern = r'(?:.+?\(.+?\)-.+?'
collect = []
for idx, row in chunk.iterrows():
    
    products = row['Device Name(Brand Name)-Intended Use']
    #rint("Original String:", products)
    
    # Find all matches
    matches = re.findall(pattern, products)
    #rint("Matches:", matches)
    #rint(len(matches))
    for match in matches:
        new_row = []
        device_name = match[0]
        brand_name = match[1]
        use = match[2]
        new_row.append(row['S. No.'])
        new_row.append(device_name)
        new_row.append(brand_name)
        new_row.append(use)
        collect.append(new_row)
        
        
    #rint("$$$$$$$$$$$$$$$$$$$$$$$$$")



In [132]:
export = pd.DataFrame(collect)
export.columns = ['S. No.','Device Name','Brand Name','Use']
export

,S. No.,Device Name,Brand Name,Use
0,1,Urethral Catheter,GPC Urethral Catheter,"It is used for urine drainage,Suction Catheter..."
1,1,".,Chest/Thoracic Drainage Catheter-It is used ...",GPC Foley Balloon Catheter,It is a urinary tract catheter used for short ...
2,1,".,IV Flow Regulator",GPC IV Flow Regulator,It is used to deliver accurate volume of IV fl...
3,1,".,IV Infusion Set",GPC IV Infusion Set,The Infusion Sets are used only for gravity fe...
4,1,".,Yankaur Suction Set",GPC Yankaur Suction Set,It is used for removal of secretion and blood ...
...,...,...,...,...
49378,10951,".,Shaft Screw",GPC Shaft Screw,"Used for attachment of implants to bone, bone ..."
49379,10951,".,DCS Barrel Double Angled Plate",GPC DCS Barrel Double Angled Plate,"Used to immobilize fractured segments,DHS Barr..."
49380,10952,Total Hip Replacement System,Pl refer Accessories/Component section,Total hip arthroplasty is intended to provide ...
49381,10953,ADHESIVE DRESSING BANDAGES FABRIC,CERDAK,"Exudate Absorption Wound dressing, Bacteroio s..."


In [133]:
export['Device Name'] = export['Device Name'].map(lambda x: x.replace(".,", ""))

In [135]:
export['Brand Name'] = export['Brand Name'].map(lambda x: x.replace(".,", ""))

In [136]:
export['Use'] = export['Use'].map(lambda x: x.replace(".,", ""))

In [137]:
export

,S. No.,Device Name,Brand Name,Use
0,1,Urethral Catheter,GPC Urethral Catheter,"It is used for urine drainage,Suction Catheter..."
1,1,Chest/Thoracic Drainage Catheter-It is used fo...,GPC Foley Balloon Catheter,It is a urinary tract catheter used for short ...
2,1,IV Flow Regulator,GPC IV Flow Regulator,It is used to deliver accurate volume of IV fl...
3,1,IV Infusion Set,GPC IV Infusion Set,The Infusion Sets are used only for gravity fe...
4,1,Yankaur Suction Set,GPC Yankaur Suction Set,It is used for removal of secretion and blood ...
...,...,...,...,...
49378,10951,Shaft Screw,GPC Shaft Screw,"Used for attachment of implants to bone, bone ..."
49379,10951,DCS Barrel Double Angled Plate,GPC DCS Barrel Double Angled Plate,"Used to immobilize fractured segments,DHS Barr..."
49380,10952,Total Hip Replacement System,Pl refer Accessories/Component section,Total hip arthroplasty is intended to provide ...
49381,10953,ADHESIVE DRESSING BANDAGES FABRIC,CERDAK,"Exudate Absorption Wound dressing, Bacteroio s..."


In [139]:
del payload['Unnamed: 0']
payload

,S. No.,License Number,License Holder Name,Device Name(Brand Name)-Intended Use,Approving Authority,Device Class
0,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B
1,2,03-13-0966,TTK Healthcare Limited,Natural Rubber Latex Male Condoms(ULINZI - For...,"Central Drug Standard Control Organisation, So...",Class C
2,3,0499,QUALPRO DIAGNOSTICS A DIVISION OF TULIP DIAGNO...,25-OH VITAMIN D ELISA-The Product is intended ...,Goa,Class B
3,4,0623,CORAL CLINICAL SYSTEMS A DIVISION OF TULIP DIA...,ENZYMATIC ASSAY FOR THE QUANTITATIVE DETERMINA...,Goa,Class B
4,5,13/28/17,Dynamic Techno Medicals Pvt. Ltd,Surgical Suture(Dynalon)-Use in general soft t...,"Central Drug Standard Control Organisation, So...",Class C
...,...,...,...,...,...,...
10949,10950,S0079/15/MB,VANGUARD DIAGNOSTICS PRIVATE LIMITED,HIV Gen.4 Microwell ELISA(VDx)-HIV Gen. 4 Micr...,"CDSCO -North Zone, Ghaziabad",Class D
10950,10951,SC/02/2018,GPC Medical Limited,Locking Bolts 7.5mm Cannulated (Self Tapping)(...,"CDSCO -North Zone, Ghaziabad",Class C
10951,10952,TN00002456,TTK HEALTHCARE LIMITED (ORTHO DIVISION),Total Hip Replacement System(Pl refer Accessor...,"Central Drug Standard Control Organisation, So...",Class C
10952,10953,TN00002473,CARBORUNDUM UNIVERSAL LTD,ADHESIVE DRESSING BANDAGES FABRIC(CERDAK)-Exud...,"Central Drug Standard Control Organisation, So...",Class C


In [141]:
cdsco_in = payload.merge(export)
cdsco_in

,S. No.,License Number,License Holder Name,Device Name(Brand Name)-Intended Use,Approving Authority,Device Class,Device Name,Brand Name,Use
0,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B,Urethral Catheter,GPC Urethral Catheter,"It is used for urine drainage,Suction Catheter..."
1,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B,Chest/Thoracic Drainage Catheter-It is used fo...,GPC Foley Balloon Catheter,It is a urinary tract catheter used for short ...
2,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B,IV Flow Regulator,GPC IV Flow Regulator,It is used to deliver accurate volume of IV fl...
3,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B,IV Infusion Set,GPC IV Infusion Set,The Infusion Sets are used only for gravity fe...
4,1,02/SCof2018,GPC Medical Limited,Urethral Catheter(GPC Urethral Catheter)-It is...,Uttar Pradesh,Class B,Yankaur Suction Set,GPC Yankaur Suction Set,It is used for removal of secretion and blood ...
...,...,...,...,...,...,...,...,...,...
49378,10951,SC/02/2018,GPC Medical Limited,Locking Bolts 7.5mm Cannulated (Self Tapping)(...,"CDSCO -North Zone, Ghaziabad",Class C,Shaft Screw,GPC Shaft Screw,"Used for attachment of implants to bone, bone ..."
49379,10951,SC/02/2018,GPC Medical Limited,Locking Bolts 7.5mm Cannulated (Self Tapping)(...,"CDSCO -North Zone, Ghaziabad",Class C,DCS Barrel Double Angled Plate,GPC DCS Barrel Double Angled Plate,"Used to immobilize fractured segments,DHS Barr..."
49380,10952,TN00002456,TTK HEALTHCARE LIMITED (ORTHO DIVISION),Total Hip Replacement System(Pl refer Accessor...,"Central Drug Standard Control Organisation, So...",Class C,Total Hip Replacement System,Pl refer Accessories/Component section,Total hip arthroplasty is intended to provide ...
49381,10953,TN00002473,CARBORUNDUM UNIVERSAL LTD,ADHESIVE DRESSING BANDAGES FABRIC(CERDAK)-Exud...,"Central Drug Standard Control Organisation, So...",Class C,ADHESIVE DRESSING BANDAGES FABRIC,CERDAK,"Exudate Absorption Wound dressing, Bacteroio s..."


In [142]:
cdsco_in.to_excel("CDSCO IN.xlsx")

In [16]:
# Extract the components into new columns
grill[['Device Name', 'Brand Name', 'Intended Use']] = grill['Device Name(Brand Name)-Intended Use'].str.extract(pattern)

In [17]:
grill

,Unnamed: 0,S. No.,License Number,License Holder Name,Device Name(Brand Name)-Intended Use,Approving Authority,Device Class,Device Name,Brand Name,Intended Use
4124,24,4125,IMP/MD/2020/000507,Mani Medical India Private Limited,Disposable Skin Stapler(Manipler)-To be used f...,CDSCO,Class B,Disposable Skin Stapler,Manipler,To be used for skin suture in general surgery ...
10803,3,10804,MFG/MD/2023/000986,"SURGILIFE MEDICAL DEVICES PVT. LTD.,","COTTON GRUDGES,GAUZE & BANDAGES(LIFEMED, SMDPL...",East Delhi Division,Class B,"COTTON GRUDGES,GAUZE & BANDAGES","LIFEMED, SMDPL","A strip of fabric used especially to cover, dr..."
10227,27,10228,MFG/MD/2023/000499,XCELLANCE MEDICAL TECHNOLOGIES PVT. LTD.,Suction and Irrigation Pump(Shalya Aquaflow)-S...,Kokan Division,Class B,Suction and Irrigation Pump,Shalya Aquaflow,Shalya Aquaflow Suction Irrigation Pump is use...


In [20]:
import re
# Step 1: Split the column values into separate rows
# Split by comma and stack the result to create a new row for each entry
s = grill['Device Name(Brand Name)-Intended Use'].str.split(',\s*', expand=True).stack().reset_index(level=1, drop=True)

# Create a new DataFrame from the stacked Series
df_expanded = pd.DataFrame(s, columns=['Expanded'])

# Step 2: Extract Device Name, Brand Name, and Intended Use into new columns
pattern = r'\'?(.+?)\((.+?)\)-(.+?)\'?'
df_final = df_expanded['Expanded'].str.extract(pattern, flags=re.IGNORECASE)
df_final.columns = ['Device Name', 'Brand Name', 'Intended Use']

print(df_final)

                        Device Name       Brand Name Intended Use
4124        Disposable Skin Stapler         Manipler            T
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803                           NaN              NaN          NaN
10803     

In [21]:
df_final

,Device Name,Brand Name,Intended Use
4124,Disposable Skin Stapler,Manipler,T
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN
10803,NaN,NaN,NaN


In [18]:
grill.to_excel("Grill IN.xlsx")